### PLAN
1. Load up to Jun 2022 into BigQuery
2. Create github action to run BQ insert script nightly
3. Set grouped database table (history_df) in BigQuery to refresh nightly after insert
4. Make script to pull data history_df from BigQuery data

8. For streamlit app append [history_df, current_df, future_df] to make the combo_df
    - put history_df into cache using st.experimental_memo so its only called once each time app is opened

In [2]:
# Initialize python libs & SQL creads
from datetime import datetime, timedelta, date
import pandas as pd
from pandasql import sqldf
import streamlit as st
import pull_nrg_data
import json
import http.client
import certifi
import ssl
import os
import timeit

In [ ]:
pull_nrg_data.release_token(accessToken)

In [16]:
# Release token
import certifi
import ssl
import http.client
server = 'api.nrgstream.com'

def release_token(accessToken):
    path = '/api/ReleaseToken'
    headers = {'Authorization': f'Bearer {accessToken}'}
    context = ssl.create_default_context(cafile=certifi.where())
    conn = http.client.HTTPSConnection(server,context=context)
    conn.request('DELETE', path, None, headers)
    res = conn.getresponse()
    print('token released')

accessToken = 'GI3R5KS9k5PMXiD0s-iE69JDcH1T98THkIiTR3LXEjwvzGF86jaN_lZ8tL08_FsdM4ZpGYjFtRhnyvxvJoQMhkNiwRI6ybVAy22QTDhvKNGN2V0STLG2mhWiiAcZJnIVUAbbpgchsM-medLxM36QM8-5SwTOR1TwHNqKG24uWm5RDN8e2IhA1HrZ1fNGEEqCIl7-kMrw9idj5xyA9mqqXhGuK6IV9h27LTA7du08WPXOMkHqiM8Tna4Bs8bl8SALfeXdx-eA3EdRSpExjq4gShpyfXZREsQzSW5n86vMifcmqipOsesISLGRd_rYDbcmsKgoljBkATT8rORLbSplchbc9C0'
release_token(accessToken)

token released


In [ ]:
# Joining daily to hourly data
import pandas as pd

intraday = pd.DataFrame(index=pd.date_range('2016-01-01', '2016-01-07', freq='H'),data=[i for i in range(145)], columns=['hourly'])
daily = pd.DataFrame(index=pd.date_range('2016-01-01', '2016-01-07', freq='D'), data=[i for i in range(7)], columns=['daily'])
df = intraday.join(daily).fillna(method='ffill')
df